#### Overview
- Reproducing https://medium.com/@dave.cote.msc/hybrid-multimodal-neural-network-architecture-combination-of-tabular-textual-and-image-inputs-7460a4f82a2e
- This script inspects the datasets (images)

In [2]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import datetime
import pickle

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Layer
from keras import backend as K

# other imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
import h5py
import json
import time

# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)





ModuleNotFoundError: No module named 'keras'